In [88]:
# import public packages
import os
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.patches import Rectangle
import scipy
import mne
import sys

from mne.time_frequency import tfr_morlet
from mne.baseline import rescale
from scipy.signal import spectrogram, hann, butter, filtfilt, hilbert
from scipy import signal, interpolate, stats
from scipy.interpolate import make_interp_spline, BSpline
from io import open
from importlib import reload

# import own functions
from utils import find_folders
import dat_preproc
import fix_annot_onsets
import mat2fif
import baseline_correction
import normalization

### Add Directories/ Load Files

In [89]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT


In [90]:
fname = 'sub-009_ses-2020091810045195_run-BrainSense20200918103300.mat'

raw = mne.io.read_raw_fieldtrip(
    os.path.join(
        project_path,
        'data',
        'raw_data',
        'test',
        fname
    ),
    info = None
)

subID = 'Sub009'

Creating RawArray with float64 data, n_channels=6, n_times=199187
    Range : 0 ... 199186 =      0.000 ...   796.744 secs
Ready.


c:\Users\mathiopv\Anaconda3\lib\site-packages\pymatreader\utils.py:230: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_16932\1409303202.py:3: RuntimeWarning: Importing FieldTrip data without an info dict from the original file. Channel locations, orientations and types will be incorrect. The imported data cannot be used for source analysis, channel interpolation etc.
  raw = mne.io.read_raw_fieldtrip(
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_16932\1409303202.py:3: RuntimeWarning: The supplied FieldTrip structure does not have an elec or grad field. No channel locations will extracted and the kind of channel might be inaccurate.
  raw = mne.io.read_raw_fieldtrip(
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_16932\1409303202.py:3: RuntimeWarning: Cannot guess the correct type of channel LFP_L

In [91]:
reload(dat_preproc)
%matplotlib qt
matplotlib.rc('font', size=10)
x = raw.get_data(reject_by_annotation = 'omit',picks=[0,1])
raw = raw
win_samp = 250
noverlap = 0.5
window = hann(win_samp, sym=False)
f, t, Sxx = dat_preproc.fft_rawviz(raw, x, win_samp, noverlap)

In [92]:
fft_name = str(subID)+'_FFT'
print(fft_name)

Sub009_FFT


In [93]:
#Saving the figures & files
fft_fig = os.path.join(project_path, 'figures','FFts','test/')
fft_file = os.path.join(project_path, 'data','FFTs','test/')

plt.savefig(str(fft_fig)+str(fft_name),dpi = 300)

np.save(str(fft_file)+str(fft_name)+'.npy',Sxx)

In [94]:
#Saving the raw data as a fif file
raw2 = raw.copy()
Draw = raw2.get_data()
info = raw.info
new_raw = mne.io.RawArray(Draw, info)

fif_name = os.path.join(project_path, 'data','Fifs','test/',str(subID)+'_FIF.fif')
new_raw.save(fif_name)


Creating RawArray with float64 data, n_channels=6, n_times=199187
    Range : 0 ... 199186 =      0.000 ...   796.744 secs
Ready.
Writing C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\Fifs\test\Sub009_FIF.fif
    Writing channel names to FIF truncated to 15 characters with remapping
Closing C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\Fifs\test\Sub009_FIF.fif
[done]


C:\Users\mathiopv\AppData\Local\Temp\ipykernel_16932\2956889989.py:8: RuntimeWarning: This filename (C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\Fifs\test\Sub009_FIF.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  new_raw.save(fif_name)


In [ ]:
''''
test_raw = mne.io.read_raw_fif(os.path.join(
    project_path,
        'data',
        'Fifs',
        'test',
        'Sub005_FIF.fif'
    )
)
'''

In [96]:
#Zscore Normalization of raw data & plot of new figure
reload(baseline_correction)
data = Sxx[1,:,]
t = t
baseline = (None, None)
raw = new_raw
stim_ch = 1
bs_data = baseline_correction.baseline_corr(data, t, baseline, raw = new_raw, stim_ch = 5)

Applying baseline correction (mode: zscore)


In [97]:
fft_fig = os.path.join(project_path, 'figures','FFts','test/')
fft_file = os.path.join(project_path, 'data','FFTs','test/')

plt.savefig(str(fft_fig)+str(fft_name)+'_zscored',dpi = 300)
np.save(str(fft_file)+str(fft_name)+'_zscored'+'.npy',bs_data)

In [98]:
d = {'onset': [1, 647, 677],
    'duration': [30, 11, 10],
    'description': ['StimOff','StimOn','Rebound']
}

epoch_df = pd.DataFrame(data = d)
print(epoch_df)

   onset  duration description
0      1        30     StimOff
1    647        11      StimOn
2    677        10     Rebound


In [99]:
## Print Power Spectra with Raw Data
reload(dat_preproc)
filt_dat = raw.get_data()
time_onsets = epoch_df
window = 250
noverlap = 0.5
ylim2 = 0.2

fig = plt.figure(figsize = (12,5))

plt.subplot(1,2,1)
title1 = str(subID)+ '_LSTN'
ps1 = dat_preproc.epoch_ps(filt_dat, time_onsets, window, noverlap, 0, 0.1, title1)

plt.subplot(1,2,2)
title2 = str(subID)+ '_RSTN'
ps2 = dat_preproc.epoch_ps(filt_dat, time_onsets, window, noverlap, 1, 0.2, title2)

plt.show()

In [100]:
## Printing the Power Spectra (RAW)
ps_all = np.concatenate((ps1,ps2), axis = 0)
ps_df = pd.DataFrame(np.transpose(ps_all),
    columns = ['StimOff_LSTN','StimOn_LSTN','Rebound_LSTN',
    'StimOff_RSTN','StimOn_RSTN','Rebound_RSTN'],
    )
print(ps_df)

     StimOff_LSTN  StimOn_LSTN  Rebound_LSTN  StimOff_RSTN  StimOn_RSTN  \
0        0.996756     1.321940      1.106753      1.429674     2.076187   
1        3.720128     6.424341      4.990033      5.184370     9.624407   
2        3.739745     5.557906      6.709074      2.842197     6.349065   
3        3.863152     3.409129      3.110431      2.245529     3.749034   
4        3.214321     1.472868      1.482851      2.327703     4.753713   
..            ...          ...           ...           ...          ...   
121      0.051184     0.106431      0.047157      0.043892    61.977601   
122      0.000749     0.001719      0.000960      0.000637     0.135418   
123      0.000405     0.000757      0.000407      0.000365     0.007241   
124      0.000268     0.000583      0.000332      0.000218     0.002918   
125      0.000148     0.000224      0.000266      0.000122     0.000717   

     Rebound_RSTN  
0        0.485717  
1        3.929979  
2        2.006347  
3        2.847171  

In [101]:
#Saving the figures & files
fft_fig = os.path.join(project_path, 'figures','Power_Spectra','test/')
fft_file = os.path.join(project_path, 'data','power_spectra','test/')

plt.savefig(str(fft_fig)+str(fft_name)+'_PS',dpi = 150)
ps_df.to_csv(str(fft_file)+str(fft_name)+'_PS.csv')

In [102]:
## Print Power Spectra with Zscored Data
fig = plt.figure(figsize = (5,5))
ps_m1s0 = np.mean(bs_data[:,epoch_df.onset[0]:epoch_df.onset[0]+ epoch_df.duration[0]],1)
ps_m1s1 = np.mean(bs_data[:,epoch_df.onset[1]:epoch_df.onset[1]+ epoch_df.duration[1]],1)
ps_reb = np.mean(bs_data[:,epoch_df.onset[2]:epoch_df.onset[2]+ epoch_df.duration[2]],1)

plt.plot(np.arange(1,127),ps_m1s0,label = epoch_df.description[0])
plt.plot(np.arange(1,127),ps_m1s1,label = epoch_df.description[1])
plt.plot(np.arange(1,127),ps_reb,label = epoch_df.description[2])

plt.legend()

plt.xlim(60,90)
plt.xlabel('Frequency [Hz]')
plt.ylabel('LFP Power (z-scored)')
plt.show()

ps_allzscored = np.transpose(np.squeeze(np.array([[ps_m1s0], [ps_m1s1],[ps_reb]])))
ps_allzscored_df = pd.DataFrame(ps_allzscored,
    columns = ['StimOff','StimOn','Rebound'],
    )
print(ps_allzscored_df)

      StimOff    StimOn   Rebound
0    0.463046  0.862231 -0.119793
1    0.280632  0.969436  0.086032
2   -0.069517  0.551016 -0.217419
3   -0.100006  0.377245  0.090971
4   -0.064832  0.768466  0.415465
..        ...       ...       ...
121 -1.073133  2.674426 -1.072499
122 -1.036090  2.728912 -1.005469
123 -0.729480  1.599501 -0.720972
124 -0.305274  1.503159 -0.297383
125 -0.300852  0.780453 -0.237188

[126 rows x 3 columns]


In [103]:
plt.savefig(str(fft_fig)+str(fft_name)+'_zscoredPS',dpi = 150)
ps_allzscored_df.to_csv(str(fft_file)+str(fft_name)+'_zscoredPS.csv')

### Average all Power Spectra and Plot them

In [ ]:
all_ps = pd.read_csv(os.path.join(
    project_path,
        'data',
        'power_spectra',
        'test',
        'AllPS_GammaIpsi.csv'
))

In [ ]:
all_ps_mean = np.mean(all_ps,1)
all_ps_sem = stats.sem(all_ps_mean)

In [ ]:
plt.plot(np.arange(1,127), all_ps_mean)
#plt.fill_between(np.arange(1,127), all_ps_mean-all_ps_sem, all_ps_mean+all_ps_sem, alpha = 0.2)
plt.xlim(50, 90)
plt.ylim(0, 0.1)